# Capstone Project Jupyter Notebook

In [4]:
import pandas as pd
import numpy as np

In [13]:
# Load data from the url and only pick the first tabel as a Data Frame
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [18]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# All neighbourhood for the same Postal code has been concated into a single row
data = data[data['Borough']!= 'Not assigned']
data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
# There is no cell with Not assigned neighborhood 
data[data['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [22]:
print(data.shape)

(103, 3)
